In [ ]:
#@title Define if we are on Colab and mount drive { display-mode: "form" }
try:
  from google.colab import drive
  drive.mount('/content/gdrive')
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
#@title (COLAB ONLY) Clone GitHub repo { display-mode: "form" }

if IN_COLAB:
  from getpass import getpass
  user = getpass('GitHub user')
  password = getpass('GitHub password')
  import os
  os.environ['GITHUB_AUTH'] = user + ':' + password
  !git clone https://$GITHUB_AUTH@github.com/lluissalord/radiology_ai.git

  %cd radiology_ai

In [1]:
#@title Setup environment and Colab general variables { display-mode: "form" }
%%capture
%run conda_colab.ipynb

In [3]:
import os

from fastai.basics import *
from fastai.medical.imaging import *

In [4]:
PATH_PREFIX = os.path.join(PATH_DRIVER, DATA_FOLDER, '')
raw_folder = PATH_PREFIX + 'DICOMS'
organize_folder = PATH_PREFIX + 'pending_classification'
doubt_folder = PATH_PREFIX + 'doubt'
preprocess_folder = PATH_PREFIX + 'preprocess'

correct_check_DICOM_dict = {
    # 'SeriesDescription': ['RODILLA AP', 'RODILLAS AP'],
    'SeriesDescription': ['RODILLA AP'],
    'BodyPartExamined': ['LOWER LIMB', 'KNEE']
}

doubt_check_DICOM_dict = {
    'SeriesDescription': ['PIERNA AP MESA']
}

metadata_labels_path = os.path.join(PATH_PREFIX, 'metadata_labels.csv')
if os.path.exists(metadata_labels_path):
    metadata_labels = pd.read_csv(metadata_labels_path, index_col=1)
else:
    metadata_labels = None

In [5]:
from utils import organize_folders, generate_template, rename_patient, move_blocks

groups = ['Daniel_Berlanga', 'David_Codina', 'Federico_Yanez', 'Guillem_Navarro', 'Jose_Colombo', 'Raul_Franco']
subgroup_length = 20
relation_filepath = PATH_PREFIX + 'relation.csv'
filename_prefix = 'IMG_'
copy = False

In [ ]:
relation_df = organize_folders(raw_folder, organize_folder, relation_filepath, reset=True, groups=groups, subgroup_length=subgroup_length, filename_prefix=filename_prefix, force_extension='.dcm', copy=copy, metadata_labels=metadata_labels, check_DICOM_dict=correct_check_DICOM_dict, debug=False)

In [ ]:
_ = organize_folders(raw_folder, doubt_folder, relation_filepath, reset=True, groups=groups, subgroup_length=subgroup_length, filename_prefix=filename_prefix, force_extension='.dcm', copy=copy, check_DICOM_dict=doubt_check_DICOM_dict, debug=False)

In [ ]:
dicom_files = get_files(organize_folder, extensions='.dcm')
rename_patient(dicom_files)

In [ ]:
generate_template(organize_folder, groups, subgroup_length, filename_prefix=filename_prefix, excel=True)

In [ ]:
move_blocks(organize_folder, new_folder='Test_participant', blocks=[31,32,30,29], relation_filepath=relation_filepath, template_extension='xlsx')

In [8]:
from utils import open_name_relation_file, move_relation

# Open relation file where the move/copy will be based on
relation_df = open_name_relation_file(relation_filepath, sep=',')

move_relation(relation_filepath, copy=True, to_raw=False)

100%|██████████| 646/646 [00:47<00:00, 13.70it/s]
